In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import eppy_multi
from multiprocess import Pool

In [5]:
n_processes = 22
all_runs = eppy_multi.build_runs(n_processes)

('historical', 2020)
('ssp126', 2035)
('ssp126', 2050)
('ssp126', 2065)
('ssp126', 2080)
('ssp245', 2035)
('ssp245', 2050)
('ssp245', 2065)
('ssp245', 2080)
('ssp585', 2035)
('ssp585', 2050)
('ssp585', 2065)
('ssp585', 2080)


In [7]:
len(all_runs[0])*22

3344

In [8]:
if __name__ ==  '__main__': 

    with Pool(processes = n_processes) as p: 
        results = p.map_async(eppy_multi.wrap_run_loop, all_runs)
        p.close()
        p.join()


In [99]:
flat_runs = [item for sublist in all_runs for item in sublist]
strip_mall = [f for f in flat_runs if 'StripMall' in f['file_name']]
eppy_multi.wrap_run_loop(strip_mall)

C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\post1980\idf_files\v22_2\2a_post1980_v22_2\RefBldgStripMallPost1980_V22-2-0_2A_USA_TX_HOUSTON.idf
C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\post1980\results\v22_2\2a\historical_2020\RefBldgStripMallPost1980
C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\post1980\idf_files\v22_2\2a_post1980_v22_2\RefBldgStripMallPost1980_V22-2-0_2A_USA_TX_HOUSTON.idf
C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\post1980\results\v22_2\2a\ssp126_2035\RefBldgStripMallPost1980
C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\post1980\idf_files\v22_2\2a_post1980_v22_2\RefBldgStripMallPost1980_V22-2-0_2A_USA_TX_HOUSTON.idf
C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\post1980\results\v22_2\2a\ssp126_2050\RefBldgStripMallPost1980
C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\post1980\idf_fil

'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\post1980\\results\\v22_2\\2a\\ssp585_2065\\RefBldgStripMallPost1980'

In [ ]:
import os
import glob
# from honeybee_energy import run as hbe
from eppy.modeleditor import IDF
import sys

In [ ]:
vintage = 'pre1980'
cz = '1a'

In [ ]:
project_root = '/Users/jmccarty/GitHub/cmip6_and_buildings'
# idf_dir = glob.glob(os.path.join(project_root,'eplus_simulations',vintage,'idf_files','v22_2',f'{cz}*'))[0]
idf_path = "/Users/jmccarty/Desktop/test_idf_run/test_idf.idf"
epw = glob.glob(os.path.join(project_root,'weather_files','tmy3',f'{cz}*'))[0]
# hbe.run_idf(idf_file_path=idf, epw_file_path=epw, )



In [ ]:
idf_file = "/Users/jmccarty/GitHub/cmip6_and_buildings/eplus_simulations/pre1980/idf_files/v22_2/1a_pre1980_v22_2/RefBldgFullServiceRestaurantPre1980_V22-2-0_1A_USA_FL_MIAMI.idf"

def decomponse_idf_filename(idf_file):
    sim_dir_split = idf_file.split("eplus_simulations")
    file_segments = sim_dir_split[1].split(os.sep)
    idf_segments = file_segments[-1].split("_")

    return {'project_root': sim_dir_split[0],
            'vintage': file_segments[1],
            'version': file_segments[3],
            'building_type': idf_segments[0],
            'climate_zone': idf_segments[2].lower()}
    
decomponse_idf_filename(idf_file)

In [ ]:
a = glob.glob(os.path.join(project_root,'eplus_simulations',vintage,'idf_files','v22_2',f'{cz}*'))[0]
# os.path.dirname(a)
idf_fname = glob.glob(os.path.join(a,'*.idf'))[0]
cz_dir = os.path.dirname(idf_fname).split(os.sep)[-1]

In [ ]:

def decompose_idf_filename(idf_file):
    sim_dir_split = idf_file.split("eplus_simulations")
    file_segments = sim_dir_split[1].split(os.sep)
    idf_segments = file_segments[-1].split("_")

    details = {'project_root': sim_dir_split[0],
            'vintage': file_segments[1],
            'version': file_segments[3],
            'building_type': idf_segments[0],
            'climate_zone': idf_segments[2].lower()
            }
    return details


def create_results_dir(idf):
    fname = idf.idfname
    idf_details = decompose_idf_filename(fname)

    results_pardir = os.path.join(idf_details['project_root'],
                                  'eplus_simulations',
                                  idf_details['vintage'],
                                  idf_details['version'],
                                  idf_details['climate_zone']
                                  )
    # check_dir_create(results_pardir)

    results_dir = os.path.join(results_pardir, idf_details['building_type'])
    # check_dir_create(results_dir)

    return results_dir

In [ ]:
create_results_dir(idf)

In [ ]:
def get_year_bands(year, formatted=True):
    if formatted:
        year_bands = f'{year-15}-{year+15}'
    else:
        year_bands = (year-15, year+15)
    return year_bands

def get_epw_name(idf_fname, scenario, year, ptile=50):
    idf_details = decompose_idf_filename(idf_path)
    project_root = idf_details['project_root']
    cz = idf_details['climate_zone']
    
    
    if scenario=='historical':
        epw_path = glob.glob(os.path.join(project_root,'weather_files','tmy3',f'{cz}*.epw'))[0]
    else:
        year_bands = get_year_bands(year, formatted=True)
        epw_path = os.path.join(project_root,'weather_files','morphed',cz,scenario,'EPWs',f'{scenario}_{ptile}_{year_bands}.epw')
    return epw_path

scenarios = ['historical', 'ssp126', 'ssp245', 'ssp585']
years = [2020, 2035, 2050, 2065, 2080]
for scenario in scenarios:
    for year in years:
        if scenario=='historical':
            year = 2020
        else:
            if year==2020:
                continue
        epw_path = get_epw_name(idf_path, scenario, year)
        print(epw_path, os.path.exists(epw_path))



In [ ]:
project_root = "/Users/jmccarty/GitHub/cmip6_and_buildings"
vintages = ['pre1980','post1980', 'new']

def load_idf_list(vintages):
    all_idf = []
    for vintage in vintages:
        for par_dir in glob.glob(os.path.join(project_root, 'eplus_simulations', vintage, 'idf_files', 'v22_2','*v22_2')):
            idf_files = glob.glob(os.path.join(par_dir,'*.idf'))
            all_idf.append(idf_files)
    
    return [item for sublist in all_idf for item in sublist]


In [ ]:
idf_path = "/Users/jmccarty/GitHub/cmip6_and_buildings/eplus_simulations/post1980/idf_files/v22_2/3a_post1980_v22_2/RefBldgPrimarySchoolPost1980_V22-2-0_3A_USA_GA_ATLANTA.idf"

idf = IDF(idf_path, epw)

idfversion = idf.idfobjects['version'][0].Version_Identifier.split('.')
idfversion.extend([0] * (3 - len(idfversion)))
idfversionstr = '-'.join([str(item) for item in idfversion])
fname = idf.idfname
options = {
    'ep_version': idfversionstr,  # runIDFs needs the version number
    'output_prefix': os.path.basename(fname).split('.')[0],
    'output_suffix': 'D',
    'output_directory': create_results_dir(idf),
    'readvars': True,
    'expandobjects': True
}
options

In [ ]:
results_path = os.path.join(os.path.dirname(idf_path),'results')
if os.path.exists(results_path):
    pass
else:
    os.mkdir(results_path)
os.chdir(results_path)

idf = IDF(idf_path, epw)
idf.run()

In [ ]:
help(idf.run)

# Results

In [4]:
import os
import glob

In [15]:
project_root = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings"

vintage = 'new'
climate_zone = '1a'
scen_year = 'ssp126_2035'
building_type = 'RefBldgMidriseApartment'
cz_dir = os.path.join(project_root,'eplus_simulations',vintage,'results','v22_2',climate_zone)

results_dir = glob.glob(os.path.join(cz_dir, scen_year, building_type+"*"))[0]

glob.glob(os.path.join(results_dir,"*.csv"))
    # \historical_2020\RefBldgFullServiceRestaurantPre1980\RefBldgFullServiceRestaurantPre1980_V22-2-0_2A_USA_TX_HOUSTON.csv"

['C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\results\\v22_2\\1a\\ssp126_2035\\RefBldgMidriseApartmentNew2004\\RefBldgMidriseApartmentNew2004_V22-2-0_1A_USA_FL_MIAMI-meter.csv',
 'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\results\\v22_2\\1a\\ssp126_2035\\RefBldgMidriseApartmentNew2004\\RefBldgMidriseApartmentNew2004_V22-2-0_1A_USA_FL_MIAMI-ssz.csv',
 'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\results\\v22_2\\1a\\ssp126_2035\\RefBldgMidriseApartmentNew2004\\RefBldgMidriseApartmentNew2004_V22-2-0_1A_USA_FL_MIAMI-zsz.csv',
 'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\results\\v22_2\\1a\\ssp126_2035\\RefBldgMidriseApartmentNew2004\\RefBldgMidriseApartmentNew2004_V22-2-0_1A_USA_FL_MIAMI.csv']

# Check IDF Updater

In [106]:
from multiprocess import Pool
import update_idf_v1 as update_idf
import sys
import os 
import glob
sys.platform

'win32'

In [149]:
project_root = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings"

pre1980_3b_dir = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\pre1980\idf_files\v7_2\refbldg_3b_usa_nv_las_vegas_pre1980_v1-4_7-2"
post1980_3b_dir = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\post1980\idf_files\v7_2\refbldg_3b_usa_nv_las_vegas_post1980_v1-4_7-2"
new_3b_dir = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\new\idf_files\v7_2\refbldg_3b_usa_nv_las_vegas_new2004_v1-4_7-2"

pre1980_7_2 = glob.glob(os.path.join(pre1980_3b_dir,'*.idf'))
post1980_7_2 = glob.glob(os.path.join(post1980_3b_dir,'*.idf'))
new_7_2 = glob.glob(os.path.join(new_3b_dir,'*.idf'))
idf_patch_list = [pre1980_7_2, post1980_7_2, new_7_2]
bldg_group_patch_list = ['pre1980','post1980','new']

cz = '3b'
run_bool = False
patch_inputs = []

for n, bldg_group in enumerate(bldg_group_patch_list):
    bldg_group_dir = os.path.join(project_root, 'eplus_simulations', bldg_group)
    temp_dir = os.path.join(bldg_group_dir,'idf_files','v7_2', f'temp_{cz}')
    dest_dir = os.path.join(bldg_group_dir,'idf_files','v22_2',f'{cz}_{bldg_group}_v22_2')
    if len(glob.glob(os.path.join(dest_dir,"*")))>0:
        existing_idfs = glob.glob(os.path.join(dest_dir,"*"))
    for idf in idf_patch_list[n]:
        bldg_type = idf.split(os.sep)[-1].lower().split(bldg_group)[0]
        exist_sum_check = [1 for ex in existing_idfs if bldg_type in ex.lower()]
        if sum(exist_sum_check)>0:
            pass
        else:
            input_tuple_patch = (run_bool, idf, temp_dir, dest_dir)
            patch_inputs.append(input_tuple_patch)
        

In [111]:
chunks = update_idf.chunk_into_n(patch_inputs, 16)


In [114]:
chunks[15]

[(True,
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_files\\v7_2\\refbldg_3b_usa_nv_las_vegas_new2004_v1-4_7-2\\RefBldgStripMallNew2004_v1.4_7.2_3B_USA_NV_LAS_VEGAS.idf',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_files\\v7_2\\temp_3b',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_files\\v22_2\\3b_new_v22_2'),
 (True,
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_files\\v7_2\\refbldg_3b_usa_nv_las_vegas_new2004_v1-4_7-2\\RefBldgSuperMarketNew2004_v1.4_7.2_3B_USA_NV_LAS_VEGAS.idf',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_files\\v7_2\\temp_3b',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_files\\v22_2\\3b_new_v22_2'),
 (True,
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_fi